# ETL Process Tomography Data

In [4]:
import os
import re
import glob
import pandas as pd

First create regex for finding all csv files with specified format.

sincos_file_format = '1210619_04_30_00.csv'
snr_file_format = '1210618_20_40_00_SNR.csv'
stack_file_format = '1210619_04_10_00to1210619_00_00_00.csv'
std_file_format = '1210619_04_20_00to1210619_00_00_00_std.csv'



In [7]:
regex_date      = '([0-9]{6}_[0-9]{2}_[0-9]{2}_[0-9]{2})'
regex_sincos    = re.compile('(?P<station>[0-9])' + regex_date + '.csv')
regex_snr       = re.compile('(?P<station>[0-9])' + regex_date + '_SNR.csv')
regex_stack     = re.compile('(?P<station>[0-9])' + regex_date + 'to' +
                             '(?P<station2>[0-9])' + regex_date + '.csv')
regex_std       = re.compile('(?P<station>[0-9])' + regex_date + 'to' +
                             '(?P<station2>[0-9])' + regex_date + '_std.csv')

In [8]:
def get_files(filepath, regex):
    all_files = []
    count_match = 0
    count_not_match = 0

    for root, dirs, files in os.walk(filepath):
        glob_files = glob.glob(os.path.join(root, '*.csv'))
        for f in glob_files:
            head, tail = os.path.split(f)
            if(regex.match(tail)):
                all_files.append(os.path.abspath(f))
                count_match +=1
            else:
                count_not_match+=1
    
    print(count_match)
    print(count_not_match)
    return all_files

# Process Max SNR/Standard Deviation Files

In [12]:
std_files = get_files('data/historical_data', regex_std)

std_files

45
140


['C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_20_40_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_20_50_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_21_00_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_21_10_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_21_20_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_21_30_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_21_40_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_21_50_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_22_00_00to1210618_00_00_00_std.csv',
 'C:\\workspace\\tomo\\tomo-etl\\data\\historical_data\\1210618_22_10_00to1210618_00_00_00_

In [15]:
std_df = pd.read_csv(std_files[0], sep=' ')
std_df.head()

,Date_l,Time_l,snr_l
0,7.051157,34.88855,3.453090
1,7.051157,34.80505,3.294046
2,7.051157,34.67020,3.326672
3,7.051157,34.68335,3.274864
4,7.051157,34.76050,3.329226


Change date into 